In [1]:
from androguard.misc import AnalyzeAPK
a, d, dx = AnalyzeAPK("../data/droidkungfu_w_benign/5/benign/87057929EFD04326DA2A553FF91B9027AF1AFDE0A6168046F18966F950CA0AF5.apk")

In [2]:
ba, bd, bdx = AnalyzeAPK("../data/droidkungfu_w_benign/0_app_fileman/benign/FFE44A8695CCEB4979825C54A29B4CB9765057308A97E49F6C6C5C473DED0AB3.apk")

In [3]:
ma, md, mdx = AnalyzeAPK("../data/droidkungfu_w_benign/0_app_fileman/malicious/1CA20CC2CCF0E981781B25BAD314098A168B6FAF848393A37D19A302A40F3F4C.apk")

In [4]:
# x = dx.get_class_analysis(classes[1])

In [5]:
fields = dx.get_fields()
print(fields)

<generator object Analysis.get_fields at 0x7f93cbdaf6d0>


In [6]:
def get_opcodes(dx):
    ops = []
    for method in dx.get_methods():
        if method.is_external():
            continue
        # Need to get the EncodedMethod from the MethodClassAnalysis object
        m = method.get_method()
        if m.get_code():
            # get_code() returns None or a DalvikCode object
            # get_bc() returns a DCode object
            # get_raw() returns bytearray
            ops.append(m.get_code().get_bc().get_raw())
            
    return ops

In [7]:
def get_method_counts(dx):
    from collections import defaultdict
    from operator import itemgetter
    c = defaultdict(int)

    for method in dx.get_methods():
        if method.is_external():
            continue
        m = method.get_method()
        for ins in m.get_instructions():
            c[(ins.get_op_value(), ins.get_name())] += 1

    for k, v in sorted(c.items(), key=itemgetter(1), reverse=True)[:10]:
        print(k, '-->',  v)
    
    return c

In [8]:
m_ops = get_opcodes(mdx)
b_ops = get_opcodes(bdx)
print(len(m_ops), len(b_ops))

1152 77


In [9]:
m_count_stats = get_method_counts(mdx)
print("\n")
b_count_stats = get_method_counts(bdx)

(110, 'invoke-virtual') --> 3960
(12, 'move-result-object') --> 2869
(26, 'const-string') --> 2603
(113, 'invoke-static') --> 1687
(84, 'iget-object') --> 1525
(112, 'invoke-direct') --> 1314
(18, 'const/4') --> 1294
(10, 'move-result') --> 1048
(34, 'new-instance') --> 937
(56, 'if-eqz') --> 853


(110, 'invoke-virtual') --> 267
(12, 'move-result-object') --> 213
(26, 'const-string') --> 146
(84, 'iget-object') --> 102
(112, 'invoke-direct') --> 98
(10, 'move-result') --> 92
(113, 'invoke-static') --> 87
(18, 'const/4') --> 85
(56, 'if-eqz') --> 73
(20, 'const') --> 67


In [10]:
def get_ast(dx):
    from androguard.decompiler.dad.decompile import DvMethod
    ret = {}
    for method in dx.get_methods():
        if method.is_external():
            continue
        dv = DvMethod(dx.get_method(method.get_method()))
        dv.process(doAST=True)
        ret[method] = dv.get_ast()
    return ret

In [11]:
m_ast = get_ast(mdx)
print(m_ast.keys())

dict_keys([<analysis.MethodClassAnalysis Lbander/fileman/DialogFactory$1;-><init>(Lbander/fileman/DialogFactory;)V [access_flags=constructor] @ 0xf538>, <analysis.MethodClassAnalysis Lbander/fileman/DialogFactory$1;->onClick(Landroid/content/DialogInterface; I)V [access_flags=public] @ 0xf554>, <analysis.MethodClassAnalysis Lbander/fileman/DialogFactory$2;-><init>(Lbander/fileman/DialogFactory;)V [access_flags=constructor] @ 0xf5b4>, <analysis.MethodClassAnalysis Lbander/fileman/DialogFactory$2;->onClick(Landroid/content/DialogInterface; I)V [access_flags=public] @ 0xf5d0>, <analysis.MethodClassAnalysis Lbander/fileman/DialogFactory$3;-><init>(Lbander/fileman/DialogFactory;)V [access_flags=constructor] @ 0xf630>, <analysis.MethodClassAnalysis Lbander/fileman/DialogFactory$3;->onClick(Landroid/content/DialogInterface; I)V [access_flags=public] @ 0xf64c>, <analysis.MethodClassAnalysis Lbander/fileman/DialogFactory;->access$0(Lbander/fileman/DialogFactory;)Landroid/content/Context; [acces

In [12]:
bxg = bdx.get_call_graph()

In [13]:
b_nodes = bxg.nodes(data=True)

In [14]:
def get_method_for_class(dx, className):
    try:
        class_obj = dx.classes[className]
        for meth in class_obj.get_methods():
            print("usage of method {}".format(meth.name))
            for _, call, _ in meth.get_xref_from():
                print("  called by -> {} -- {}".format(call.class_name, call.name))
    except Exception:
        pass 

In [15]:
def get_method_for_all_classes(dx):
    for cls in dx.get_classes():
        className = cls.orig_class.get_name()
        print(f"---------------------{className}----------------------")
        get_method_for_class(bdx, className)

In [16]:
get_method_for_all_classes(bdx)

---------------------Lbander/fileman/FileUtils;----------------------
usage of method <init>
usage of method copyFile
  called by -> Lbander/fileman/Main$2; -- onClick
usage of method readTextFile
  called by -> Lbander/fileman/TextViewer; -- onResume
---------------------Lbander/fileman/Main$1;----------------------
usage of method <init>
  called by -> Lbander/fileman/Main; -- onOptionsItemSelected
usage of method onClick
---------------------Lbander/fileman/Main$2;----------------------
usage of method <init>
  called by -> Lbander/fileman/Main; -- onOptionsItemSelected
usage of method onClick
---------------------Lbander/fileman/Main$3;----------------------
usage of method <init>
  called by -> Lbander/fileman/Main; -- onContextItemSelected
usage of method onClick
---------------------Lbander/fileman/Main$FileComparer;----------------------
usage of method <init>
  called by -> Lbander/fileman/Main; -- listDirectory
usage of method compare
usage of method compare
  called by -> Lb

In [17]:
get_method_for_all_classes(mdx)

---------------------Lbander/fileman/DialogFactory$1;----------------------
---------------------Lbander/fileman/DialogFactory$2;----------------------
---------------------Lbander/fileman/DialogFactory$3;----------------------
---------------------Lbander/fileman/DialogFactory;----------------------
---------------------Lbander/fileman/FileUtils;----------------------
usage of method <init>
usage of method copyFile
  called by -> Lbander/fileman/Main$2; -- onClick
usage of method readTextFile
  called by -> Lbander/fileman/TextViewer; -- onResume
---------------------Lbander/fileman/KeyConstants;----------------------
---------------------Lbander/fileman/Main$1;----------------------
usage of method <init>
  called by -> Lbander/fileman/Main; -- onOptionsItemSelected
usage of method onClick
---------------------Lbander/fileman/Main$2;----------------------
usage of method <init>
  called by -> Lbander/fileman/Main; -- onOptionsItemSelected
usage of method onClick
---------------------

usage of method getEntry
  called by -> Lbander/fileman/Preferences; -- setTextSizeSummary
  called by -> Lbander/fileman/Preferences; -- setSortOrderSummary
usage of method setSummary
  called by -> Lbander/fileman/Preferences; -- setSortOrderSummary
  called by -> Lbander/fileman/Preferences; -- setTextSizeSummary
usage of method setOnPreferenceChangeListener
  called by -> Lbander/fileman/Preferences; -- onCreate
  called by -> Lbander/fileman/Preferences; -- onCreate
usage of method setValue
  called by -> Lbander/fileman/Preferences; -- onPreferenceChange
  called by -> Lbander/fileman/Preferences; -- onPreferenceChange
---------------------Landroid/preference/Preference;----------------------
usage of method getKey
  called by -> Lbander/fileman/Preferences; -- onPreferenceChange
---------------------Landroid/widget/EditText;----------------------
usage of method getText
  called by -> Lbander/fileman/Properties$1; -- onClick
  called by -> Lbander/fileman/TextViewer; -- onPause


In [44]:
print(ma.get_requested_permissions())

['android.permission.WRITE_EXTERNAL_STORAGE', 'android.permission.INTERNET', 'android.permission.ACCESS_NETWORK_STATE', 'android.permission.RECEIVE_BOOT_COMPLETED', 'android.permission.ACCESS_COARSE_LOCATION', 'android.permission.ACCESS_FINE_LOCATION', 'android.permission.READ_PHONE_STATE']
